<a href="https://colab.research.google.com/github/RoboTuan/FPAR/blob/master/save_Cams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
import tensorflow as tf
import torchvision
import numpy as np
from torchvision import transforms
import cv2

from PIL import Image


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q "./drive/My Drive/GTEA61.zip"
!rm -rf "./__MACOSX"

In [4]:
!rm -rf "./ML_DL_Project"

In [5]:
import os 
if not os.path.isdir('./ML_DL_Project'):
  !git clone https://github.com/RoboTuan/ML_DL_Project.git

Cloning into 'ML_DL_Project'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 1357 (delta 136), reused 116 (delta 62), pack-reused 1161
Receiving objects: 100% (1357/1357), 85.77 MiB | 41.64 MiB/s, done.
Resolving deltas: 100% (947/947), done.


In [50]:
from ML_DL_Project.Scripts.objectAttentionModelConvLSTM import *

from ML_DL_Project.Scripts.attentionMapModel import attentionMap
from ML_DL_Project.Scripts.twoStreamModel import *
from ML_DL_Project.Scripts.attentionModelLSTA import *

In [7]:
base_dataset = "./GTEA61"

if not os.path.isdir('./GTEA61_val'):
  os.makedirs("./GTEA61_val")
  os.makedirs("./GTEA61_val/flow_x_processed")
  os.makedirs("./GTEA61_val/flow_y_processed")
  os.makedirs("./GTEA61_val/processed_frames2")
  pass


for directory in sorted(os.listdir(base_dataset)):
  if not directory.startswith('.'):
    directory_path = os.path.join(base_dataset, directory)
    new_directory_path = os.path.join("./GTEA61_val", directory)

    for user in sorted(os.listdir(directory_path)):
      if not user.startswith('.') and user == "S2":
        current_folder = os.path.join(directory_path, user)
        !cp -r "{current_folder}" "{new_directory_path}"
        !rm -rf "{current_folder}"
        print(current_folder)


./GTEA61/flow_x_processed/S2
./GTEA61/flow_y_processed/S2
./GTEA61/processed_frames2/S2


In [8]:
!rm -rf 'CAMs'

In [9]:
  if not os.path.isdir('./CAMs'):
    os.makedirs('./CAMs')

In [61]:

def takeError(examinedClass,model_path,model_type,selfsup_bool,regression_bool,twoStream,LSTA,Flow):
  stackSize=1
  Dataset = []
  Labels = []
  NumFrames = []
  directory = examinedClass
  # print for debugging
  print(f"directory: {directory}")
  action = directory.split('/')[-1]
  if not os.path.isdir(f'./CAMs/{model_type}'):
    os.makedirs(f'./CAMs/{model_type}')
    os.makedirs(f'./CAMs/{model_type}/' + action)
  elif not os.path.isdir(f'./CAMs/{model_type}/'+ action):
    os.makedirs(f'./CAMs/{model_type}/' + action)

  
  for target in sorted(os.listdir(directory)):
    #print(f"target: {target}")
    if not target.startswith('.'):
      #print(target)
      directory1 = os.path.join(directory, target)
      # print for debugging
      #print(f"directory1: {directory1}")
      insts = sorted(os.listdir(directory1))
      # print for debugging
      #print(f"insts: {insts}")
      #insts = insts + "/rgb"
      inst_dir= os.path.join(directory1,insts[1])
      #print(f"inst_dir {inst_dir}")


      for image in sorted(os.listdir(inst_dir)):
        image_path= os.path.join(inst_dir, image)
        #print(f"image_path :{image_path}")

        ###################################################################
        num_classes = 61 # Classes in the pre-trained model
        mem_size = 512
        model_state_dict = model_path # Weights of the pre-trained model

        if (selfsup_bool == True):
          model = SelfSupAttentionModel(num_classes=num_classes, mem_size=mem_size, REGRESSOR=regression_bool,Flow=Flow)
          model.load_state_dict(torch.load(model_state_dict))
          model_backbone = model.resNet      
        else:
          if (twoStream==True):
            model= twoStreamAttentionModel(LSTA=LSTA, c_cam_classes=100)
            model.load_state_dict(torch.load(model_state_dict))
            model_backbone = model.frameModel.resNet
            
          else:
            if (LSTA==True):
              model=attentionModelLSTA(num_classes=61,c_cam_classes=100)
            else:
              model = attentionModel(num_classes=num_classes, mem_size=mem_size)
            model.load_state_dict(torch.load(model_state_dict))
            model_backbone = model.resNet
       
        
        attentionMapModel = attentionMap(model_backbone).cuda()
        attentionMapModel.train(False)
        for params in attentionMapModel.parameters():
            params.requires_grad = False
        ###################################################################

        normalize = transforms.Normalize(
          mean=[0.485, 0.456, 0.406],
          std=[0.229, 0.224, 0.225]
        )
        preprocess1 = transforms.Compose([
          transforms.Scale(256),
          transforms.CenterCrop(224),
        ])

        preprocess2 = transforms.Compose([
            transforms.ToTensor(),
            normalize])


        fl_name_in = image_path
        #print(f"fl_name_in :{fl_name_in}")
        fl_name_out = "./CAMs/{}/{}/{}_{}.jpg".format(model_type,action, str(target),fl_name_in.split('/')[-1].split('.')[0])
        #print(f"fl_name_out :{fl_name_out}")
        img_pil = Image.open(fl_name_in)
        img_pil1 = preprocess1(img_pil)
        img_size = img_pil1.size
        size_upsample = (img_size[0], img_size[1])
        img_tensor = preprocess2(img_pil1)
        img_variable = Variable(img_tensor.unsqueeze(0).cuda())
        img = np.asarray(img_pil1)
        attentionMap_image = attentionMapModel(img_variable, img, size_upsample)
        cv2.imwrite(fl_name_out, attentionMap_image)



## RGB MODEL

In [55]:

!unzip -q "./drive/MyDrive/Runs/Parte 1/7FramesRGB_Attention.zip"
!rm -rf "./__MACOSX"

replace 7FramesRGB_Attention/close_honey_rgb_attention1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [12]:
!rm -rf "./CAMs/RGB_7"

In [57]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./7FramesRGB_Attention/model_rgb_state_dict(1).pth',model_type='RGB_7',
          selfsup_bool=False,regression_bool=False,twoStream=False,LSTA=False) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./7FramesRGB_Attention/model_rgb_state_dict(1).pth',model_type='RGB_7',
          selfsup_bool=False,regression_bool=False,twoStream=False,LSTA=False) #save cams of the class that is always selected instead of the previous one

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


##RGB MODEL 16 FRAMES

In [58]:
!unzip -q "./drive/MyDrive/Runs/Parte 1/16FramesRGB_Attention.zip" 
!rm -rf "./__MACOSX"

In [59]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./16FramesRGB_Attention/model_rgb_state_dict (1).pth',model_type='RGB_16',
          selfsup_bool=False,regression_bool=False,twoStream=False,LSTA=False) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./16FramesRGB_Attention/model_rgb_state_dict (1).pth',model_type='RGB_16',
          selfsup_bool=False,regression_bool=False,twoStream=False, LSTA=False) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


##TWO STREAM 7 FRAMES

In [27]:
!unzip -q "./drive/MyDrive/Runs/Parte 1/TwoStream7Frames.zip" 
!rm -rf "./__MACOSX"

In [40]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./TwoStream7Frames/model_twoStream_state_dict.pth',model_type='TwoStream_7',
          selfsup_bool=False,regression_bool=False,twoStream=True, LSTA=False) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./TwoStream7Frames/model_twoStream_state_dict.pth',model_type='TwoStream_7',
          selfsup_bool=False,regression_bool=False, twoStream=True,LSTA=False) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


##TWO STREAM 16 FRAMES

In [41]:
!unzip -q "./drive/MyDrive/Runs/Parte 1/TwoStream16Frames.zip" 
!rm -rf "./__MACOSX"

In [42]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./TwoStream16Frames/model_twoStream_state_dict.pth',model_type='TwoStream_16',
          selfsup_bool=False,regression_bool=False,twoStream=True, LSTA=False) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./TwoStream16Frames/model_twoStream_state_dict.pth',model_type='TwoStream_16',
          selfsup_bool=False,regression_bool=False, twoStream=True,LSTA=False) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


##TWO STREAM LSTA 7 FRAMES

In [43]:
!unzip -q "./drive/MyDrive/Runs/Variations/LSTA/cartella_LSTA7frames.zip" 
!rm -rf "./__MACOSX"

In [46]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./cartella_LSTA7frames/outDir/gtea61/twoStream_LSTA/model_twoStream_state_dict.pth',model_type='TwoStreamLSTA_7',
          selfsup_bool=False,regression_bool=False,twoStream=True, LSTA=True) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./cartella_LSTA7frames/outDir/gtea61/twoStream_LSTA/model_twoStream_state_dict.pth',model_type='TwoStreamLSTA_7',
          selfsup_bool=False,regression_bool=False, twoStream=True,LSTA=True) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


## TWO STREAM LSTA 16 FRAMES

In [47]:
!unzip -q "./drive/MyDrive/Runs/Variations/LSTA/twoStream_LSTA_16.zip" 
!rm -rf "./__MACOSX"

In [48]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./twoStream_LSTA_16/model_twoStream_state_dict.pth',model_type='TwoStreamLSTA_16',
          selfsup_bool=False,regression_bool=False,twoStream=True, LSTA=True) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./twoStream_LSTA_16/model_twoStream_state_dict.pth',model_type='TwoStreamLSTA_16',
          selfsup_bool=False,regression_bool=False, twoStream=True,LSTA=True) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


##RGB LSTA 7 FRAMES

In [49]:
!unzip -q "./drive/MyDrive/Runs/Variations/LSTA/rgb_lsta_7.zip" 
!rm -rf "./__MACOSX"

In [52]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./rgb_lsta_7/stage2/model_rgb_lsta_state_dict.pth',model_type='RGB_LSTA_7',
          selfsup_bool=False,regression_bool=False,twoStream=False, LSTA=True) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./rgb_lsta_7/stage2/model_rgb_lsta_state_dict.pth',model_type='RGB_LSTA_7',
          selfsup_bool=False,regression_bool=False, twoStream=False,LSTA=True) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


##RGB LSTA 16 FRAMES

In [53]:
!unzip -q "./drive/MyDrive/Runs/Variations/LSTA/rgb_lsta_16.zip" 
!rm -rf "./__MACOSX"

In [54]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./rgb_lsta_16/stage2/model_rgb_lsta_state_dict.pth',model_type='RGB_LSTA_16',
          selfsup_bool=False,regression_bool=False,twoStream=False, LSTA=True) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./rgb_lsta_16/stage2/model_rgb_lsta_state_dict.pth',model_type='RGB_LSTA_16',
          selfsup_bool=False,regression_bool=False, twoStream=False,LSTA=True) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


## SELF SUP MODEL

In [19]:
from ML_DL_Project.Scripts.SelfSupObjectAttentionModelConvLSTM import *

In [21]:
!rm -rf "./outDir"
!rm -rf "./CAMs"

In [22]:
!unzip -q "./drive/MyDrive/Runs/Parte 2/No Regression/Decay rate optimization/cartella_selfSupDecayRate0,5_prova2.zip"
!rm -rf "./__MACOSX"

In [23]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./outDir/gtea61/selfSup/stage2/model_selfSup_state_dict.pth',model_type='self_sup',
          selfsup_bool=True,regression_bool=False) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./outDir/gtea61/selfSup/stage2/model_selfSup_state_dict.pth',model_type='self_sup',
          selfsup_bool=True,regression_bool=False) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread
Regression:  False
FLow:  False


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regression:  False
FLow:  False
Regressi

## SELF SUP REGRESSION MODEL

In [24]:
from ML_DL_Project.Scripts.SelfSupObjectAttentionModelConvLSTM import *

In [17]:
!unzip -q "./drive/MyDrive/Runs/Parte 2/Regression/Decay rate optimization/cartella_RegselfSupGirodecayRate0.3_prova1.zip" 
!rm -rf "./__MACOSX"

In [26]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./cartella_RegselfSupGirodecayRate0.3_prova1/outDir/gtea61/regSelfSup/stage2/model_selfSup_state_dict.pth',model_type='self_sup_regr',
          selfsup_bool=True,regression_bool=True) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./cartella_RegselfSupGirodecayRate0.3_prova1/outDir/gtea61/regSelfSup/stage2/model_selfSup_state_dict.pth',model_type='self_sup_regr',
          selfsup_bool=True,regression_bool=True) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread
Regression:  True
FLow:  False


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regression:  True
FLow:  False
Regressi

##SELF SUP FLOW 

In [60]:
!unzip -q "./drive/MyDrive/Runs/Variations/Self sup Warp Flow/Regression/Decay rate/cartella_DecayRate0.3prova_2.zip" 
!rm -rf "./__MACOSX"

In [62]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./cartella_DecayRate0.3prova_2/outDir/gtea61/regSelfSup_flow/stage2/model_selfSup_state_dict.pth',model_type='self_sup_flow',
          selfsup_bool=True,regression_bool=True,twoStream=False,LSTA=False,Flow=True) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./cartella_DecayRate0.3prova_2/outDir/gtea61/regSelfSup_flow/stage2/model_selfSup_state_dict.pth',model_type='self_sup_flow',
          selfsup_bool=True,regression_bool=True,twoStream=False,LSTA=False,Flow=True) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread
Regression:  True
FLow:  True


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression:  True
FLow:  True
Regression

In [63]:
!zip -r "CAMs" "./CAMs"

  adding: CAMs/ (stored 0%)
  adding: CAMs/RGB_7/ (stored 0%)
  adding: CAMs/RGB_7/open_chocolate/ (stored 0%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0023.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0024.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0013.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0022.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0012.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0011.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0021.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0006.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0016.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0008.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0004.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0002.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0025.jpg (deflated 1%)
  adding: CAMs/RGB_7/open_chocolate/1_rgb0014.jpg (deflat